In [58]:
%%bash

pip install --upgrade pip
pip install farm-haystack[colab,preprocessing,elasticsearch,inference]

In [59]:
%%bash

wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
chown -R daemon:daemon elasticsearch-7.9.2

In [60]:
%%bash --bg

sudo -u daemon -- elasticsearch-7.9.2/bin/elasticsearch

In [61]:
import time

time.sleep(30)

In [62]:
import os
from haystack.document_stores import ElasticsearchDocumentStore

document_store = ElasticsearchDocumentStore(host="localhost", port = "9200", username="", password="", index="document")

In [63]:
from haystack.utils import fetch_archive_from_http

doc_dir = "/content/data"

In [64]:
from haystack import Pipeline
from haystack.nodes import TextConverter, PreProcessor

indexing_pipeline = Pipeline()
text_converter = TextConverter()
preprocessor = PreProcessor(
    clean_whitespace=True,
    clean_header_footer=True,
    clean_empty_lines=True,
    split_by="word",
    split_length=200,
    split_overlap=20,
    split_respect_sentence_boundary=True,
)

In [65]:
import os

indexing_pipeline.add_node(component=text_converter, name="TextConverter", inputs=["File"])
indexing_pipeline.add_node(component=preprocessor, name="PreProcessor", inputs=["TextConverter"])
indexing_pipeline.add_node(component=document_store, name="DocumentStore", inputs=["PreProcessor"])

In [66]:
files_to_index = [doc_dir + "/" + f for f in os.listdir(doc_dir)]
indexing_pipeline.run_batch(file_paths=files_to_index)

Preprocessing: 100%|██████████| 53/53 [00:04<00:00, 11.77docs/s]


{'documents': [<Document: {'content': '\ufeffГОСТ 413-91 (ИСО 1420-87) Ткани с резиновым или пластмассовым покрытием. Определение водонепроницаемостиПрименяется с 01.07.1992 взамен ГОСТ 413-75 Страница      ГОСТ 413-91 (ИСО 1420-87) \xa0\xa0\xa0\xa0\xa0Группа Л69  \xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0МЕЖГОСУДАРСТВЕННЫЙ СТАНДАРТ ТКАНИ С РЕЗИНОВЫМ ИЛИ ПЛАСТМАССОВЫМ ПОКРЫТИЕМ Определение водонепроницаемости Rubber or plastics coated fabrics. Determination of resistance to penetration by water ОКСТУ 2566 Дата введения 1992-07-01 ИНФОРМАЦИОННЫЕ ДАННЫЕ 1. РАЗРАБОТАН И ВНЕСЕН Загорским филиалом ВНИИЭМИ2. УТВЕРЖДЕН И ВВЕДЕН В ДЕЙСТВИЕ Постановлением Комитета стандартизации и метрологии СССР от 27.06.91 N 11723. Настоящий стандарт разработан методом прямого применения международного стандарта ИСО 1420-87* "Ткани с резиновым или пластмассовым покрытием. ', 'content_type': 'text', 'score': None, 'meta': {'_split_id': 0, '_split_overlap': [{'doc_id': '9831f5a0c659523904ce8c5edbe64f19', 'range':

In [67]:
from haystack.nodes import BM25Retriever

retriever = BM25Retriever(document_store=document_store)

In [68]:
from haystack.nodes import FARMReader

reader = FARMReader(model_name_or_path="cointegrated/rubert-tiny2", use_gpu=True)
#reader = FARMReader(model_name_or_path="AlexKay/xlm-roberta-large-qa-multilingual-finedtuned-ru", use_gpu=True)
#reader = FARMReader(model_name_or_path="abletobetable/mdeberta-ru-qa", use_gpu=True)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [69]:
from haystack import Pipeline

querying_pipeline = Pipeline()
querying_pipeline.add_node(component=retriever, name="Retriever", inputs=["Query"])
querying_pipeline.add_node(component=reader, name="Reader", inputs=["Retriever"])

In [74]:
prediction = querying_pipeline.run(
    query="Аппаратура для проведения испытания обувь", params={"Retriever": {"top_k": 1}, "Reader": {"top_k": 1}}
)

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 95.59 Batches/s]


In [75]:
from pprint import pprint

pprint(prediction)

{'answers': [<Answer {'answer': 'верхом из натуральной, искусственной и синтетической кожи, текстиля, с комбинированным верхом и устанавливает метод определения прочности крепления подошв в обуви клеевого, литьевого, котловой и прессовой вулканизации и комбинированных методов крепления, а также накладок и подметок. Требования настоящего стандарта являются обязательными. (Измененная редакция, Изм. N 2, 3). 1. МЕТОД ОТБОРА ОБРАЗЦОВ', 'type': 'extractive', 'score': 0.006266128271818161, 'context': 'верхом из натуральной, искусственной и синтетической кожи, текстиля, с комбинированным верхом и устанавливает метод определения прочности крепления подошв в обуви клеевого, литьевого, котловой и прессовой вулканизации и комбинированных методов крепления, а также накладок и подметок. Требования настоящего стандарта являются обязательными. (Измененная редакция, Изм. N 2, 3). 1. МЕТОД ОТБОРА ОБРАЗЦОВ', 'offsets_in_document': [{'start': 71, 'end': 471}], 'offsets_in_context': [{'start': 0, 'end': 4

3. Simplify the printed answers:

In [76]:
from haystack.utils import print_answers

print_answers(prediction, details="minimum")  ## Choose from `minimum`, `medium` and `all`

'Query: Аппаратура для проведения испытания обувь'
'Answers:'
[   {   'answer': 'верхом из натуральной, искусственной и синтетической кожи, '
                  'текстиля, с комбинированным верхом и устанавливает метод '
                  'определения прочности крепления подошв в обуви клеевого, '
                  'литьевого, котловой и прессовой вулканизации и '
                  'комбинированных методов крепления, а также накладок и '
                  'подметок. Требования настоящего стандарта являются '
                  'обязательными. (Измененная редакция, Изм. N 2, 3). 1. МЕТОД '
                  'ОТБОРА ОБРАЗЦОВ',
        'context': 'верхом из натуральной, искусственной и синтетической кожи, '
                   'текстиля, с комбинированным верхом и устанавливает метод '
                   'определения прочности крепления подошв в обуви клеевого, '
                   'литьевого, котловой и прессовой вулканизации и '
                   'комбинированных методов крепления, а такж

In [77]:
from haystack.utils import print_documents
print_documents(prediction)


Query: Аппаратура для проведения испытания обувь

{   'content': '(ИУС 2-87, 8-89, 10-91) Настоящий стандарт распространяется на '
               'обувь с верхом из натуральной, искусственной и синтетической '
               'кожи, текстиля, с комбинированным верхом и устанавливает метод '
               'определения прочности крепления подошв в обуви клеевого, '
               'литьевого, котловой и прессовой вулканизации и комбинированных '
               'методов крепления, а также накладок и подметок. Требования '
               'настоящего стандарта являются обязательными. (Измененная '
               'редакция, Изм. N 2, 3). 1. МЕТОД ОТБОРА ОБРАЗЦОВ 1.1. Отбор '
               'образцов - по ГОСТ 9289. 2. АППАРАТУРА 2.1. Для проведения '
               'испытания применяют разрывные машины марки РТ-250 или другие, '
               'обеспечивающие скорость движения нижнего зажима (100±10) '
               'мм/мин. Мощность машины не должна превышать нагрузку '
               'раз